In [15]:
import sqlite3
import pandas as pd

In [16]:
# Kết nối SQLite trong bộ nhớ (RAM)
conn = sqlite3.connect(":memory:")
cursor = conn.cursor()

In [17]:
# Tạo bảng student
cursor.execute("""
CREATE TABLE student (
    student_id INTEGER PRIMARY KEY,
    name TEXT,
    class TEXT,
    course_id INTEGER,
    score REAL
);
""")

# Dữ liệu cho bảng student
student_data = [
    (1, "Nguyen Minh Hoang", "May Tinh", 12, 6.7),
    (2, "Tran Thi Lan", "Kinh Te", 34, 9.2),
    (3, "Pham Van Nam", "Toan Tin", None, 7.9),
    (4, "Le Thanh Huyen", "Toan Tin", 20, 7.2),
    (5, "Vu Quoc Anh", "May Tinh", 24, 8.0),
    (6, "Dang Thuy Linh", "May Tinh", 24, 5.5),
    (7, "Bui Tien Dung", "Kinh Te", 34, 9.2),
    (8, "Ho Ngoc Mai", "Toan Tin", 20, 8.8),
    (9, "Duong Huu Phuc", "Kinh Te", None, 7.2),
    (10, "Cao Thi Hanh", "May Tinh", None, 7.0),
]

cursor.executemany("INSERT INTO student VALUES (?, ?, ?, ?, ?);", student_data)

# Tạo bảng course
cursor.execute("""
CREATE TABLE course (
    id INTEGER PRIMARY KEY,
    course_name TEXT
);
""")

# Dữ liệu cho bảng course
course_data = [
    (12, "Giai tich"),
    (34, "Thong ke"),
    (26, "Tin hoc")
]

cursor.executemany("INSERT INTO course VALUES (?, ?);", course_data)

conn.commit()

# Ý 1

In [18]:
# Tích Descartes (CROSS JOIN)
query = """
SELECT * FROM student CROSS JOIN course;
"""
df_cartesian = pd.read_sql_query(query, conn)
print("Tích Descartes (CROSS JOIN):")
print(df_cartesian)

# INNER JOIN
query = """
SELECT student.*, course.course_name
FROM student
JOIN course ON student.course_id = course.id;
"""
df_inner = pd.read_sql_query(query, conn)
print("\nINNER JOIN:")
print(df_inner)

# LEFT JOIN
query = """
SELECT student.*, course.course_name
FROM student
LEFT JOIN course ON student.course_id = course.id;
"""
df_left = pd.read_sql_query(query, conn)
print("\nLEFT JOIN:")
print(df_left)

# RIGHT JOIN (SQLite không hỗ trợ RIGHT JOIN trực tiếp, ta dùng LEFT JOIN đảo ngược)
query = """
SELECT course.*, student.*
FROM course
LEFT JOIN student ON student.course_id = course.id;
"""
df_right = pd.read_sql_query(query, conn)
print("\nRIGHT JOIN:")
print(df_right)

# FULL OUTER JOIN (SQLite không hỗ trợ trực tiếp, ta dùng UNION của LEFT JOIN và RIGHT JOIN)
query = """
SELECT student.*, course.course_name
FROM student
LEFT JOIN course ON student.course_id = course.id
UNION
SELECT student.*, course.course_name
FROM student
RIGHT JOIN course ON student.course_id = course.id;
"""
df_full_outer = pd.read_sql_query(query, conn)
print("\nFULL OUTER JOIN:")
print(df_full_outer)

# Đóng kết nối SQLite
conn.close()


Tích Descartes (CROSS JOIN):
    student_id               name     class  course_id  score  id course_name
0            1  Nguyen Minh Hoang  May Tinh       12.0    6.7  12   Giai tich
1            1  Nguyen Minh Hoang  May Tinh       12.0    6.7  26     Tin hoc
2            1  Nguyen Minh Hoang  May Tinh       12.0    6.7  34    Thong ke
3            2       Tran Thi Lan   Kinh Te       34.0    9.2  12   Giai tich
4            2       Tran Thi Lan   Kinh Te       34.0    9.2  26     Tin hoc
5            2       Tran Thi Lan   Kinh Te       34.0    9.2  34    Thong ke
6            3       Pham Van Nam  Toan Tin        NaN    7.9  12   Giai tich
7            3       Pham Van Nam  Toan Tin        NaN    7.9  26     Tin hoc
8            3       Pham Van Nam  Toan Tin        NaN    7.9  34    Thong ke
9            4     Le Thanh Huyen  Toan Tin       20.0    7.2  12   Giai tich
10           4     Le Thanh Huyen  Toan Tin       20.0    7.2  26     Tin hoc
11           4     Le Thanh Huyen  

- Tích Descartes (CROSS JOIN)
    *  Số bản ghi = Số dòng của student × Số dòng của course.
    *  Kết quả thu được là tất cả các cặp có thể có giữa hai bảng, không quan tâm đến mối liên hệ giữa chúng.
    *  Điều này dẫn đến nhiều bản ghi không có ý nghĩa thực tế, vì không phải sinh viên nào cũng học tất cả các khóa học
- INNER JOIN
    *  Chỉ giữ lại những sinh viên có course_id trùng với id trong bảng course.
    *  Các sinh viên không đăng ký khóa học nào sẽ bị loại khỏi kết quả.
- LEFT JOIN
    *  Giữ tất cả sinh viên từ bảng student, ngay cả khi họ không có khóa học tương ứng (course_id = NULL).
    *  Các sinh viên chưa đăng ký khóa học sẽ có giá trị NULL trong course_name.
- RIGHT JOIN 
    *  Giữ tất cả khóa học từ bảng course, ngay cả khi không có sinh viên nào đăng ký.
    *  Nếu một khóa học không có sinh viên, các giá trị liên quan đến sinh viên sẽ là NULL.
- FULL OUTER JOIN (Mô phỏng bằng UNION của LEFT JOIN và RIGHT JOIN)
    *  Kết hợp cả LEFT JOIN và RIGHT JOIN: giữ tất cả sinh viên và tất cả khóa học, ngay cả khi không có dữ liệu khớp.
    *  Các bản ghi có thể có giá trị NULL ở cả hai bảng nếu không tìm thấy dữ liệu tương ứng.
    
Kết luận chung:
- INNER JOIN hữu ích khi chỉ cần dữ liệu có liên kết hợp lệ.
- LEFT JOIN phù hợp khi cần danh sách sinh viên đầy đủ.
- RIGHT JOIN hữu ích khi muốn kiểm tra khóa học chưa có sinh viên.
- FULL OUTER JOIN cho cái nhìn toàn cảnh về dữ liệu.
- Tích Descartes hiếm khi được sử dụng vì số lượng bản ghi quá lớn.

# Ý 2

In [20]:
import sqlite3
import pandas as pd

# Kết nối SQLite (tạo database trong bộ nhớ)
conn = sqlite3.connect(":memory:")
cursor = conn.cursor()

# Tạo bảng student
cursor.execute('''
CREATE TABLE student (
    student_id INTEGER PRIMARY KEY,
    name TEXT,
    class TEXT,
    course_id INTEGER,
    score REAL
)
''')

# Tạo bảng course
cursor.execute('''
CREATE TABLE course (
    id INTEGER PRIMARY KEY,
    course_name TEXT
)
''')

# Chèn dữ liệu vào bảng student
student_data = [
    (1, "Nguyen Minh Hoang", "May Tinh", 12, 6.7),
    (2, "Tran Thi Lan", "Kinh Te", 34, 9.2),
    (3, "Pham Van Nam", "Toan Tin", None, 7.9),  # Bị thiếu course_id
    (4, "Le Thanh Huyen", "Toan Tin", 20, 7.2),  # course_id không tồn tại
    (5, "Vu Quoc Anh", "May Tinh", 24, 8.0),
    (6, "Dang Thuy Linh", "May Tinh", 24, 5.5),
    (7, "Bui Tien Dung", "Kinh Te", 34, 9.2),
    (8, "Ho Ngoc Mai", "Toan Tin", 20, 8.8),  # course_id không tồn tại
    (9, "Duong Huu Phuc", "Kinh Te", None, 7.2),  # Bị thiếu course_id
    (10, "Cao Thi Hanh", "May Tinh", None, 7.0)  # Bị thiếu course_id
]

cursor.executemany('''
INSERT INTO student (student_id, name, class, course_id, score)
VALUES (?, ?, ?, ?, ?)
''', student_data)

# Chèn dữ liệu vào bảng course
course_data = [
    (12, "Giai tich"),
    (34, "Thong ke"),
    (26, "Tin hoc")
]
cursor.executemany('INSERT INTO course (id, course_name) VALUES (?, ?)', course_data)

# Cập nhật course_id còn thiếu bằng giá trị hợp lệ từ bảng course
cursor.execute('''
UPDATE student
SET course_id = (SELECT id FROM course ORDER BY RANDOM() LIMIT 1)
WHERE course_id IS NULL
''')

# Xóa các bản ghi có course_id không tồn tại trong bảng course
cursor.execute('''
DELETE FROM student
WHERE course_id NOT IN (SELECT id FROM course)
''')

# Truy vấn tổng số sinh viên và điểm trung bình của từng lớp
query_class_stats = '''
SELECT class, COUNT(student_id) AS total_students, AVG(score) AS avg_score
FROM student
GROUP BY class
'''
df_class_stats = pd.read_sql(query_class_stats, conn)
print(" Tổng số sinh viên và điểm trung bình của từng lớp:")
print(df_class_stats, "\n")

# Truy vấn tổng số sinh viên và điểm trung bình của từng môn học
query_course_stats = '''
SELECT course.course_name, COUNT(student.student_id) AS total_students, AVG(student.score) AS avg_score
FROM student
JOIN course ON student.course_id = course.id
GROUP BY course.course_name
'''
df_course_stats = pd.read_sql(query_course_stats, conn)
print(" Tổng số sinh viên và điểm trung bình của từng môn học:")
print(df_course_stats, "\n")

# Phân loại sinh viên theo điểm số
query_student_category = '''
SELECT student.name, student.class, student.score,
    CASE
        WHEN student.score >= 9.0 THEN 'Xuất sắc'
        WHEN student.score >= 6.0 AND student.score <= 8.9 THEN 'Tốt'
        ELSE 'Kém'
    END AS classification
FROM student
'''
df_student_category = pd.read_sql(query_student_category, conn)
print(" Phân loại sinh viên theo điểm số:")
print(df_student_category, "\n")

# Đóng kết nối
conn.close()


 Tổng số sinh viên và điểm trung bình của từng lớp:
      class  total_students  avg_score
0   Kinh Te               3   8.533333
1  May Tinh               2   6.850000
2  Toan Tin               1   7.900000 

 Tổng số sinh viên và điểm trung bình của từng môn học:
  course_name  total_students  avg_score
0   Giai tich               4        7.2
1    Thong ke               2        9.2 

 Phân loại sinh viên theo điểm số:
                name     class  score classification
0  Nguyen Minh Hoang  May Tinh    6.7            Tốt
1       Tran Thi Lan   Kinh Te    9.2       Xuất sắc
2       Pham Van Nam  Toan Tin    7.9            Tốt
3      Bui Tien Dung   Kinh Te    9.2       Xuất sắc
4     Duong Huu Phuc   Kinh Te    7.2            Tốt
5       Cao Thi Hanh  May Tinh    7.0            Tốt 



- Nhận xét:
- ĐTB theo từng lớp:
    * Lớp Kinh Tế có điểm trung bình cao nhất (8.53), có thể do nhiều sinh viên có điểm xuất sắc.
    * Lớp Máy Tính có số lượng sinh viên ít nhất (2 sinh viên) và điểm trung bình thấp nhất (6.85).
    * Lớp Toán Tin có duy nhất 1 sinh viên, nên điểm trung bình bằng điểm của sinh viên đó (7.9).

- ĐTB theo môn học:
    * Môn Thống kê có điểm trung bình cao nhất (9.2), cho thấy sinh viên học tốt môn này.
    * Môn Giải tích có điểm trung bình thấp hơn (7.2), nhưng vẫn ở mức khá.
    * Không thấy dữ liệu cho môn "Tin học" ⇒ Có thể không có sinh viên nào chọn môn này hoặc đã bị loại bỏ do dữ liệu không hợp lệ.

- Phân loại sinh viên theo điểm số:
    * Có 2 sinh viên đạt Xuất sắc (Trần Thị Lan và Bùi Tiến Dũng) với điểm 9.2.
    * Có 4 sinh viên đạt loại Tốt, không có sinh viên nào bị xếp loại Kém.
    * Không có sinh viên nào có điểm < 6.0, chứng tỏ chất lượng học tập khá tốt.


- Tổng kết chung:

    * Lớp Kinh Tế có nhiều sinh viên và đạt điểm trung bình cao nhất.
    * Môn Thống kê có điểm trung bình cao nhất (9.2), cho thấy sinh viên học tốt môn này.
    * Không có sinh viên nào bị xếp loại Kém, chất lượng học tập tương đối ổn.


# Ý 3

In [21]:
import pandas as pd
import sqlite3

# Kết nối đến database SQLite
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

# Tạo bảng student
cursor.execute('''
    CREATE TABLE student (
        student_id INTEGER PRIMARY KEY,
        name TEXT,
        class TEXT,
        course_id INTEGER,
        score REAL
    )
''')

# Tạo bảng course
cursor.execute('''
    CREATE TABLE course (
        id INTEGER PRIMARY KEY,
        course_name TEXT
    )
''')

# Chèn dữ liệu vào bảng student
students_data = [
    (1, "Nguyen Minh Hoang", "May Tinh", 12, 6.7),
    (2, "Tran Thi Lan", "Kinh Te", 34, 9.2),
    (3, "Pham Van Nam", "Toan Tin", 20, 7.9),
    (4, "Bui Tien Dung", "Kinh Te", 34, 9.2),
    (5, "Duong Huu Phuc", "Kinh Te", None, 7.2),
    (6, "Cao Thi Hanh", "May Tinh", None, 7.0)
]
cursor.executemany("INSERT INTO student VALUES (?, ?, ?, ?, ?)", students_data)

# Chèn dữ liệu vào bảng course
courses_data = [
    (12, "Giai tich"),
    (34, "Thong ke"),
    (26, "Tin hoc")
]
cursor.executemany("INSERT INTO course VALUES (?, ?)", courses_data)

conn.commit()

# Đọc dữ liệu từ bảng student
df_students = pd.read_sql_query("SELECT * FROM student", conn)

# Đọc dữ liệu từ bảng course
df_courses = pd.read_sql_query("SELECT * FROM course", conn)

# Nối bảng student với bảng course để lấy tên môn học
df = df_students.merge(df_courses, left_on="course_id", right_on="id", how="left")
df.drop(columns=["id"], inplace=True)  # Loại bỏ cột trùng lặp

# Xếp hạng sinh viên theo điểm số toàn bộ
df["rank_overall"] = df["score"].rank(ascending=False, method="min")

# Xếp hạng sinh viên theo lớp học
df["rank_by_class"] = df.groupby("class")["score"].rank(ascending=False, method="min")

# Xếp hạng sinh viên theo môn học
df["rank_by_course"] = df.groupby("course_name")["score"].rank(ascending=False, method="min")

# Xuất kết quả
print("\n Xếp hạng theo điểm số toàn bộ sinh viên:")
print(df.sort_values("rank_overall")[["name", "class", "score", "rank_overall"]])

print("\n Xếp hạng theo điểm số trong từng lớp:")
print(df.sort_values(["class", "rank_by_class"])[["name", "class", "score", "rank_by_class"]])

print("\n Xếp hạng theo điểm số trong từng môn học:")
print(df.sort_values(["course_name", "rank_by_course"])[["name", "course_name", "score", "rank_by_course"]])

#  Lấy top 3 sinh viên cao nhất và thấp nhất
top_3_highest = df.nlargest(3, "score")[["name", "class", "score"]]
top_3_lowest = df.nsmallest(3, "score")[["name", "class", "score"]]

print("\n Top 3 sinh viên có điểm cao nhất:")
print(top_3_highest)

print("\n Top 3 sinh viên có điểm thấp nhất:")
print(top_3_lowest)

# Đóng kết nối database
conn.close()



 Xếp hạng theo điểm số toàn bộ sinh viên:
                name     class  score  rank_overall
1       Tran Thi Lan   Kinh Te    9.2           1.0
3      Bui Tien Dung   Kinh Te    9.2           1.0
2       Pham Van Nam  Toan Tin    7.9           3.0
4     Duong Huu Phuc   Kinh Te    7.2           4.0
5       Cao Thi Hanh  May Tinh    7.0           5.0
0  Nguyen Minh Hoang  May Tinh    6.7           6.0

 Xếp hạng theo điểm số trong từng lớp:
                name     class  score  rank_by_class
1       Tran Thi Lan   Kinh Te    9.2            1.0
3      Bui Tien Dung   Kinh Te    9.2            1.0
4     Duong Huu Phuc   Kinh Te    7.2            3.0
5       Cao Thi Hanh  May Tinh    7.0            1.0
0  Nguyen Minh Hoang  May Tinh    6.7            2.0
2       Pham Van Nam  Toan Tin    7.9            1.0

 Xếp hạng theo điểm số trong từng môn học:
                name course_name  score  rank_by_course
0  Nguyen Minh Hoang   Giai tich    6.7             1.0
1       Tran Thi Lan    Th

- Nhận xét kết quả
    * 2 sinh viên đạt điểm cao nhất (9.2) là Trần Thị Lan và Bùi Tiến Dũng.
    * Sinh viên có điểm thấp nhất là Nguyễn Minh Hoàng (6.7).
    * Trong lớp Kinh Tế, có 2 sinh viên đứng đầu với 9.2, nhưng May Tính có điểm trung bình thấp hơn.
    * Trong môn học, môn Thống kê có điểm cao nhất, còn Giải tích có sự chênh lệch đáng kể.

# Ý 4

In [24]:
import pandas as pd
import sqlite3
from datetime import datetime, timedelta

# Kết nối đến database SQLite
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

# Tạo bảng student với cột graduation_date có kiểu TEXT để lưu datetime dưới dạng chuỗi
cursor.execute('''
    CREATE TABLE student (
        student_id INTEGER PRIMARY KEY,
        name TEXT,
        class TEXT,
        course_id INTEGER,
        score REAL,
        graduation_date TEXT
    )
''')

# Thêm dữ liệu sinh viên vào bảng student
students_data = [
    (1, "Nguyen Minh Hoang", "May Tinh", 12, 6.7, None),
    (2, "Tran Thi Lan", "Kinh Te", 34, 9.2, None),
    (3, "Pham Van Nam", "Toan Tin", 20, 7.9, None),
    (4, "Bui Tien Dung", "Kinh Te", 34, 9.2, None),
    (5, "Duong Huu Phuc", "Kinh Te", None, 7.2, None),
    (6, "Cao Thi Hanh", "May Tinh", None, 7.0, None)
]
cursor.executemany("INSERT INTO student VALUES (?, ?, ?, ?, ?, ?)", students_data)
conn.commit()

# Đọc dữ liệu từ bảng student
df_students = pd.read_sql_query("SELECT * FROM student", conn)

# Xếp hạng sinh viên theo điểm số
df_students["rank"] = df_students["score"].rank(ascending=False, method="min")

# Xác định ngày tốt nghiệp = ngày hiện tại + số hạng
current_date = datetime.now()
df_students["graduation_date"] = df_students["rank"].apply(lambda r: (current_date + timedelta(days=int(r))).strftime('%Y-%m-%d %H:%M:%S'))

# Cập nhật giá trị graduation_date vào database (dưới dạng string)
for _, row in df_students.iterrows():
    cursor.execute("UPDATE student SET graduation_date = ? WHERE student_id = ?", 
                   (row["graduation_date"], row["student_id"]))
conn.commit()

# Hiển thị kết quả
print("\n Danh sách sinh viên với ngày tốt nghiệp dự kiến:")
print(df_students[["name", "class", "score", "rank", "graduation_date"]])

# Đóng kết nối database
conn.close()



 Danh sách sinh viên với ngày tốt nghiệp dự kiến:
                name     class  score  rank      graduation_date
0  Nguyen Minh Hoang  May Tinh    6.7   6.0  2025-03-17 21:18:26
1       Tran Thi Lan   Kinh Te    9.2   1.0  2025-03-12 21:18:26
2       Pham Van Nam  Toan Tin    7.9   3.0  2025-03-14 21:18:26
3      Bui Tien Dung   Kinh Te    9.2   1.0  2025-03-12 21:18:26
4     Duong Huu Phuc   Kinh Te    7.2   4.0  2025-03-15 21:18:26
5       Cao Thi Hanh  May Tinh    7.0   5.0  2025-03-16 21:18:26


- Nhận xét về kết quả:

- Thứ hạng và ngày tốt nghiệp:

    * Sinh viên có điểm cao nhất (9.2) là Tran Thi Lan và Bui Tien Dung có thứ hạng 1 → tốt nghiệp sớm nhất vào ngày 12/03/2025.
    * Sinh viên có điểm thấp nhất (6.7) là Nguyen Minh Hoang có thứ hạng 6 → tốt nghiệp muộn nhất vào ngày 17/03/2025.
    * Những sinh viên còn lại tốt nghiệp dựa theo thứ hạng tương ứng.

- Ngày tốt nghiệp được sắp xếp hợp lý:

    * Quy tắc: Người có điểm cao hơn sẽ tốt nghiệp sớm hơn
    * Cách tính ngày tốt nghiệp (hiện tại + số thứ hạng) đảm bảo công bằng, không có xếp hạng bị trùng lặp hoặc sai lệch.

- Sinh viên có cùng điểm số:

    * Tran Thi Lan và Bui Tien Dung đều có điểm 9.2, nên họ có cùng thứ hạng 1 và tốt nghiệp cùng ngày 12/03/2025.
    * Điều này phản ánh cách xếp hạng hợp lý, không có sự thiên vị giữa các sinh viên có điểm số bằng nhau.
- Kết luận:

    * Quy trình xác định ngày tốt nghiệp hoạt động chính xác.
    * Sinh viên điểm cao hơn tốt nghiệp sớm hơn, đúng theo yêu cầu bài toán.
